In [ ]:
import numpy as np
from app import ExponentialMixtureModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score

# Ejemplo

## Creacion de los datos

In [ ]:
np.random.seed(0)
X1 = np.random.exponential(scale=1/3, size=100)
X2 = np.random.exponential(scale=1/5, size=100)
X = np.concatenate([
    np.random.exponential(scale=1/3, size=100),  
    np.random.exponential(scale=1/5, size=100) 
])

true_labels = np.concatenate([np.zeros(100), np.ones(100)])

fig, ax = plt.subplots(figsize=(10, 6))

sns.kdeplot(X, color='black', ax=ax, label='X')
sns.kdeplot(X1, color='#3D9CB5', fill=True, ax=ax, label='X1')
sns.kdeplot(X2, color='#E98C90', fill=True, ax=ax, label='X2')
ax.legend()

## Exponential Mixture Model

In [ ]:

emm = ExponentialMixtureModel(k=2,max_iter=1000)
emm.fit(X)

clusters = emm.predict(X)


## Gaussian Mixture Model

In [ ]:
gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42)
gmm.fit(X.reshape(-1,1))

# Predecimos las etiquetas de los datos
y_gmm = gmm.predict(X.reshape(-1,1))

In [ ]:
print('Accuracy:', accuracy_score(true_labels, clusters))
print('Accuracy:', accuracy_score(true_labels, y_gmm))

# Testeo

In [ ]:
# Configuración inicial
np.random.seed(0)
num_simulations = 100
size_options = [50, 100, 200]   # Ejemplo de tamaños de muestra
lambda_options = [(1/3, 1/5), (1/2, 1/4), (1/4, 1/6)]  # Diferentes combinaciones de lambdas
results = []

for size in size_options:
    for lambdas in lambda_options:
        accuracies_emm = []
        accuracies_gmm = []
        
        for _ in range(num_simulations):
            # Generar datos exponenciales con tamaños y lambdas variables
            X = np.concatenate([
                np.random.exponential(scale=lambdas[0], size=size),
                np.random.exponential(scale=lambdas[1], size=size)
            ])
            true_labels = np.concatenate([np.zeros(size), np.ones(size)])
            
            # Modelo EMM
            emm = ExponentialMixtureModel(k=2, max_iter=500)
            emm.fit(X.reshape(-1, 1))
            predicted_labels_emm = emm.predict(X.reshape(-1, 1))
            accuracy_emm = accuracy_score(true_labels, predicted_labels_emm)
            accuracies_emm.append(accuracy_emm)
            
            # Modelo GMM
            gmm = GaussianMixture(n_components=2, max_iter=500, random_state=0)
            gmm.fit(X.reshape(-1, 1))
            predicted_labels_gmm = gmm.predict(X.reshape(-1, 1))
            accuracy_gmm = accuracy_score(true_labels, predicted_labels_gmm)
            accuracies_gmm.append(accuracy_gmm)
        
        # Guardar resultados promedio para esta configuración de tamaño y lambdas
        avg_accuracy_emm = np.mean(accuracies_emm)
        avg_accuracy_gmm = np.mean(accuracies_gmm)
        results.append({
            "size": size,
            "lambda_1": lambdas[0],
            "lambda_2": lambdas[1],
            "accuracy_emm": avg_accuracy_emm,
            "accuracy_gmm": avg_accuracy_gmm
        })

# Mostrar resultados



In [ ]:
for result in results:
    print(f"Tamaño: {result['size']}, Lambda 1: {result['lambda_1']}, Lambda 2: {result['lambda_2']}")
    print(f"  Accuracy promedio EMM: {result['accuracy_emm']:.4f}")
    print(f"  Accuracy promedio GMM: {result['accuracy_gmm']:.4f}")